Experimenting Frequent Pattern (FP-Growth) algortihm 

In [1]:
import findspark
findspark.init()
import pyspark
from pyspark.mllib.fpm import FPGrowth
import re
from random import random
from pyspark import SparkContext

In [46]:
import os
port = 4040 + hash(os.getcwd().split("/")[2])%200
config = pyspark.SparkConf().set('spark.executor.memory', '512M')\
         .set('spark.cores.max', '2')\
         .set('spark.port.maxRetries','200')\
         .set('spark.ui.port', port)    
print port,     os.getcwd().split("/")[2]
sc = pyspark.SparkContext(appName="FP ingredient", master='',conf = config )

4214 rus217


In [45]:
sc.stop()

In [48]:
rddData = sc.textFile("recipes.txt")
print rddData.take(3)
# Each line of data represents a recipes, which is like:
# cuisine;ID;ingredients1;ingredients2;...

[u'greek;10259;romaine lettuce;black olives;grape tomatoes;garlic;pepper;purple onion;seasoning;garbanzo beans;feta cheese crumbles', u'southern_us;25693;plain flour;ground pepper;salt;tomatoes;ground black pepper;thyme;eggs;green tomatoes;yellow corn meal;milk;vegetable oil', u'filipino;20130;eggs;pepper;salt;mayonaise;cooking oil;green chilies;grilled chicken breasts;garlic powder;yellow onion;soy sauce;butter;chicken livers']


In [4]:
# How many different recipes does the dataset contain?
print "Total Recipes",rddData.count()


Total Recipes 39774


In [5]:
# How many different cuisine does the dataset contain?
def splitcuisine(x):
    d = x.split(';')
    key=d[0]
    return (key)

cuisine = rddData.map(splitcuisine).distinct()
print cuisine.take(10)

print "Total cuisines: ",cuisine.count()

[u'irish', u'mexican', u'chinese', u'filipino', u'moroccan', u'french', u'cajun_creole', u'british', u'russian', u'jamaican']
Total cuisines:  20


In [35]:
# Print the TOP-10 ingredients (ordered by the most frequent usage)
ingre = rddData.flatMap(lambda line:line.split(';')[2:]).map(lambda r: (r, 1)).reduceByKey(lambda a,b: a+b)
print ingre.take(25)
ingreSorted = ingre.sortBy(lambda x:x[1],ascending=False)
print "\n\n","TOP-10 ingredients:","\n\n"
for (a,b) in ingreSorted.take(10):
    print a,b

[(u'hoja santa leaves', 2), (u'chrysanthemum leaves', 2), (u'tart filling', 1), (u'macadamia nuts', 11), (u'sweetened coconut', 3), (u'tamarind', 39), (u'fresh chervil', 15), (u'sushi nori', 5), (u'baking chocolate', 3), (u'grape juice', 6), (u'frozen onion rings', 1), (u'peeled deveined shrimp', 8), (u'ground cardamom', 237), (u'hyssop', 1), (u'light tuna packed in olive oil', 5), (u'reduced fat italian dressing', 2), (u'breadfruit', 2), (u'pancetta', 177), (u'fresh leav spinach', 57), (u'vegan parmesan cheese', 4), (u'branca menta', 1), (u'mackerel', 5), (u'large curd cottage cheese', 5), (u'young coconut meat', 2), (u'grits', 184)]


TOP-10 ingredients: 


salt 18049
olive oil 7972
onions 7972
water 7457
garlic 7380
sugar 6434
garlic cloves 6237
butter 4848
ground black pepper 4785
all-purpose flour 4632


In [98]:
# Which ingredients are frequently used together? First consider all 
# cuisines. 
def newsplit(x):
    out= []
    d = x.split(';')
    print d
    d = d[2:]
    for v in d:
        if not v in out:
            out.append(v)
    return out 
 
t = rddData.map(newsplit)
print t.take(3)
print type(t)

mode = FPGrowth.train(t,minSupport=0.05, numPartitions=10)
result = mode.freqItemsets().sortBy(lambda x: x[1], False).collect()
print "Ingredients that are frequently used together, for all cuisines:"
for r in result:
    print r

[[u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles'], [u'plain flour', u'ground pepper', u'salt', u'tomatoes', u'ground black pepper', u'thyme', u'eggs', u'green tomatoes', u'yellow corn meal', u'milk', u'vegetable oil'], [u'eggs', u'pepper', u'salt', u'mayonaise', u'cooking oil', u'green chilies', u'grilled chicken breasts', u'garlic powder', u'yellow onion', u'soy sauce', u'butter', u'chicken livers']]
<class 'pyspark.rdd.PipelinedRDD'>
Ingredients that are frequently used together, for all cuisines:
FreqItemset(items=[u'salt'], freq=18048)
FreqItemset(items=[u'onions'], freq=7972)
FreqItemset(items=[u'olive oil'], freq=7971)
FreqItemset(items=[u'water'], freq=7457)
FreqItemset(items=[u'garlic'], freq=7380)
FreqItemset(items=[u'sugar'], freq=6434)
FreqItemset(items=[u'garlic cloves'], freq=6236)
FreqItemset(items=[u'butter'], freq=4847)
FreqItemset(items=[u'ground black pepper'], fre

In [80]:
# How does the top frequent item-sets differ if you consider only mexican
# cuisine?

def newsplit2(x):
    d = x.split(';')
    key = d[0]
    value = d[2:]
    return (key,value)

data = rddData.map(newsplit2)
print data.take(5)

mex = data.filter(lambda x: "mexican" in x)
print "\n\n",mex.take(2)

def keepTheValue(kv):
    out = []
    for v in kv[1]:
        if not v in out:
            out.append(v)
    return out

print "\n\n",mex.map(keepTheValue).take(2)
mexi = mex.map(keepTheValue)
mode = FPGrowth.train(mexi,minSupport=0.05, numPartitions=10)
result = mode.freqItemsets().sortBy(lambda x: x[1], False).collect()
print "Ingredients that are frequently used together, for mexican:"
for r in result:
    print r    

[(u'greek', [u'romaine lettuce', u'black olives', u'grape tomatoes', u'garlic', u'pepper', u'purple onion', u'seasoning', u'garbanzo beans', u'feta cheese crumbles']), (u'southern_us', [u'plain flour', u'ground pepper', u'salt', u'tomatoes', u'ground black pepper', u'thyme', u'eggs', u'green tomatoes', u'yellow corn meal', u'milk', u'vegetable oil']), (u'filipino', [u'eggs', u'pepper', u'salt', u'mayonaise', u'cooking oil', u'green chilies', u'grilled chicken breasts', u'garlic powder', u'yellow onion', u'soy sauce', u'butter', u'chicken livers']), (u'indian', [u'water', u'vegetable oil', u'wheat', u'salt']), (u'indian', [u'black pepper', u'shallots', u'cornflour', u'cayenne pepper', u'onions', u'garlic paste', u'milk', u'butter', u'salt', u'lemon juice', u'water', u'chili powder', u'passata', u'oil', u'ground cumin', u'boneless chicken skinless thigh', u'garam masala', u'double cream', u'natural yogurt', u'bay leaf'])]


[(u'mexican', [u'olive oil', u'purple onion', u'fresh pineapple'